# Backtesting Configuration Batch Tester

This notebook demonstrates how to use the batch testing functionality to test multiple backtest configurations at once. It allows you to test all configurations in the database or a specific subset, and visualize the results.

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Add project root to path
current_dir = os.path.dirname(os.path.abspath(''))
sys.path.append(os.path.dirname(current_dir))

# Import custom modules
from Data.Database.db import Database
from Experements.Backtesting.batch_tester import test_all_configs, test_config, compare_configs_by_date_range

# Set plotting style
plt.style.use('ggplot')
%matplotlib inline

ModuleNotFoundError: No module named 'Data'

## Database Connection

First, connect to the database to access the stored configurations.

In [ ]:
# Create database connection
db = Database()

# Check available stocks and timeframes
stocks_df = db.get_all_stocks()
timeframes_df = db.get_all_timeframes()

print("Available Stocks:")
display(stocks_df)

print("\nAvailable Timeframes:")
display(timeframes_df)

## View Stored Configurations

Let's check what configurations are available in the database.

In [ ]:
# Get all configurations
configs_df = db.get_configs()

# Display a summary of configurations
summary_cols = ['config_id', 'stock_id', 'timeframe_id', 'recognition_technique', 'n_pips', 'lookback', 'hold_period']
config_summary = configs_df[summary_cols] if all(col in configs_df.columns for col in summary_cols) else configs_df

display(config_summary.sort_values('config_id', ascending=False))

## Test a Single Configuration

Let's start by testing a single configuration.

In [ ]:
# Set test parameters
config_id = 44  # Replace with your desired configuration ID
train_start = pd.Timestamp("2024-01-01")
train_end = pd.Timestamp("2025-01-01")
test_start = pd.Timestamp("2025-01-01")
test_end = pd.Timestamp("2025-05-01")

# Run the test
result = test_config(
    db=db,
    config_id=config_id,
    train_start=train_start,
    train_end=train_end,
    test_start=test_start, 
    test_end=test_end,
    save_report=True
)

# Display the results
if result and 'summary' in result:
    summary = result['summary']
    print(f"Configuration: {summary['config_id']}")
    print(f"Stock: {summary['stock_id']}")
    print(f"Technique: {summary['technique']}")
    print(f"Total Trades: {summary['total_trades']}")
    print(f"Win Rate: {summary['win_rate']:.2f}%")
    print(f"Total Return: {summary['total_return_pct']:.2f}%")
    print(f"Sharpe Ratio: {summary['sharpe_ratio']:.2f}")
    print(f"Max Drawdown: {summary['max_drawdown']:.2f}%")
    
    # Plot equity curve if available
    if 'equity_curve' in result:
        plt.figure(figsize=(10, 6))
        plt.plot(result['equity_curve'])
        plt.title(f"Equity Curve for Configuration {config_id}")
        plt.xlabel("Time")
        plt.ylabel("Portfolio Value")
        plt.grid(True)
        plt.show()

## Test All Configurations for a Specific Stock

Now let's test all configurations for a specific stock.

In [ ]:
# Set test parameters
stock_id = 1  # Replace with your desired stock ID
train_start = pd.Timestamp("2024-01-01")
train_end = pd.Timestamp("2025-01-01")
test_start = pd.Timestamp("2025-01-01")
test_end = pd.Timestamp("2025-05-01")

# Run the tests
results = test_all_configs(
    db=db,
    stock_id=stock_id,
    train_start=train_start,
    train_end=train_end,
    test_start=test_start,
    test_end=test_end,
    save_reports=False,
    parallel=False  # Set to True for faster testing if you have multiple cores
)

# Display the summary results
if 'summary' in results and not results['summary'].empty:
    display(results['summary'][['config_id', 'technique', 'total_trades', 'win_rate', 'profit_factor', 'total_return_pct', 'sharpe_ratio']])

## Compare Configurations Across Multiple Time Periods

Let's compare how different configurations perform across multiple time periods.

In [ ]:
# Get configuration IDs for a specific stock
stock_id = 1  # Replace with your desired stock ID
configs_df = db.get_configs(stock_id=stock_id)
config_ids = configs_df['config_id'].tolist()

# If there are too many configs, limit to the first few
if len(config_ids) > 5:
    config_ids = config_ids[:5]

print(f"Testing configurations: {config_ids}")

# Create date ranges for testing (4 quarters of a year)
base_date = pd.Timestamp("2025-01-01")
start_dates = []
end_dates = []

for i in range(4):
    period_start = base_date + pd.Timedelta(days=i*90)  # 90 days = ~3 months
    period_end = period_start + pd.Timedelta(days=90)
    start_dates.append(period_start)
    end_dates.append(period_end)
    print(f"Period {i+1}: {period_start.strftime('%Y-%m-%d')} to {period_end.strftime('%Y-%m-%d')}")

# Run the comparison
comparison_df = compare_configs_by_date_range(
    db=db,
    config_ids=config_ids,
    start_dates=start_dates,
    end_dates=end_dates,
    train_period=365,  # 1 year training
    stock_id=stock_id
)

# Display the results
if not comparison_df.empty:
    display(comparison_df[['config_id', 'period', 'total_trades', 'win_rate', 'total_return_pct', 'sharpe_ratio']])

## Create a Custom Visualization

Let's create a custom visualization of the comparison results.

In [ ]:
if not comparison_df.empty and 'period' in comparison_df.columns and 'config_id' in comparison_df.columns:
    # Create a pivot table of returns
    pivot_df = comparison_df.pivot(index='config_id', columns='period', values='total_return_pct')
    
    # Add average column
    pivot_df['Average'] = pivot_df.mean(axis=1)
    
    # Sort by average performance
    pivot_df = pivot_df.sort_values('Average', ascending=False)
    
    # Create a heatmap
    plt.figure(figsize=(12, 8))
    im = plt.pcolormesh(pivot_df.values, cmap='RdYlGn', vmin=-20, vmax=20)
    plt.colorbar(im, label='Return (%)')
    
    # Set axis ticks and labels
    plt.yticks(np.arange(0.5, len(pivot_df.index)), pivot_df.index)
    plt.xticks(np.arange(0.5, len(pivot_df.columns)), pivot_df.columns, rotation=45)
    
    plt.title('Configuration Performance by Time Period')
    plt.tight_layout()
    plt.show()

## Cleanup

Close the database connection when done.

In [ ]:
# Close database connection
db.close()